# Titanic: Machine Learning from Disaster
https://www.kaggle.com/c/titanic

Firstly, make imports, read data, pick out target column and find missings in data.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

y_train = np.array(train['Survived'])
del train['Survived']

X_all = pd.concat([train, test])
print(X_all.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB
None


Almost all machine learning algorithms work with numerical vectors. Convert data to appropriate format, filling in blanks and encode categorial features. Missing may be useful information, save it in additional column.

In [2]:
X_all['age_null'] = pd.isnull(X_all['Age'])
X_all['cabin_null'] = pd.isnull(X_all['Cabin'])

X_all['isMale'] = X_all['Sex'] == 'male'

X_all.fillna(value = {'Age': X_all['Age'].mean(),
                      'Fare' : X_all['Fare'].mean(),
                      'Embarked' : X_all['Embarked'].mode()[0]},
             inplace = True)

X_all['Embarked_S'] = X_all['Embarked'] == 'S'
X_all['Embarked_C'] = X_all['Embarked'] == 'C'
X_all['Embarked_Q'] = X_all['Embarked'] == 'Q'

X_all['PClass_1'] = X_all['Pclass'] == 1
X_all['PClass_2'] = X_all['Pclass'] == 2
X_all['PClass_3'] = X_all['Pclass'] == 3

del X_all['Cabin'], X_all['PassengerId'], X_all['Name'], X_all['Ticket'], X_all['Embarked'], X_all['Sex'], X_all['Pclass']

X_train = np.array(X_all.iloc[train.index], dtype=np.dtype('float64'))
X_test = np.array(X_all.iloc[X_train.shape[0]:], dtype=np.dtype('float64'))

Scale all features with StandardScaler thus mean will be zero and standard deviation will be one.

In [3]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Try linear model - logistic regression. Pick up regulazation parameter.

In [4]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=44)
clf = LogisticRegression()
param_grid = {'C' : [2.0 ** i for i in range(-5, 10)]}

gridSearch = GridSearchCV(estimator=clf, param_grid=param_grid, cv=kfold)
gridSearch.fit(X_train, y_train)

print("C = %0.3f \nAccuracy: %0.3f (+/- %0.3f)\n" % (gridSearch.best_params_['C'],
                                                     gridSearch.best_score_,
                                                     gridSearch.cv_results_['std_test_score'][gridSearch.best_index_] * 2))

C = 0.062 
Accuracy: 0.804 (+/- 0.115)



So, our baseline is ** 0.804 (+/- 0.115) **

Now try SVM with RBF.

In [5]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=44)
clf = SVC()
param_grid = {'C' : [2.0 ** i for i in range(-5, 10)], 'kernel': ['rbf']}

gridSearch = GridSearchCV(estimator=clf, param_grid=param_grid, cv=kfold)
gridSearch.fit(X_train, y_train)

print("C = %0.3f \nAccuracy: %0.3f (+/- %0.3f)\n" % (gridSearch.best_params_['C'],
                                                     gridSearch.best_score_,
                                                     gridSearch.cv_results_['std_test_score'][gridSearch.best_index_] * 2))

C = 2.000 
Accuracy: 0.825 (+/- 0.083)



Results improve to ** 0.825 (+/- 0.083) **

Maybe it means that data have a cluster structure. 
Try K-nearest neighbor.

In [6]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=44)
clf = KNeighborsClassifier()
param_grid = {'n_neighbors' : range(1, 26)}

gridSearch = GridSearchCV(estimator=clf, param_grid=param_grid, cv=kfold)
gridSearch.fit(X_train, y_train)

print("n_neighbors = %d \nAccuracy: %0.3f (+/- %0.3f)\n" % (gridSearch.best_params_['n_neighbors'],
                                                            gridSearch.best_score_,
                                                            gridSearch.cv_results_['std_test_score'][gridSearch.best_index_] * 2))

n_neighbors = 24 
Accuracy: 0.808 (+/- 0.088)



Results are slightly better than logistic regression: ** 0.808 (+/- 0.088) **.
Then try add information about cluster structure to features.

In [7]:
best_score = 0.0
best_n_clusters = 0

for n_clusters in range(2,101):
    kmeans = KMeans(n_clusters=n_clusters, random_state=44)
    
    X_train_cluster_dist = kmeans.fit_transform(X_train)
    X_test_cluster_dist = kmeans.transform(X_test)
    
    scaler_cluster_dist = StandardScaler()
    
    X_train_cluster_dist = scaler_cluster_dist.fit_transform(X_train_cluster_dist)
    X_test_cluster_dist = scaler_cluster_dist.transform(X_test_cluster_dist)
    
    X_train2 = np.hstack([X_train, X_train_cluster_dist])
    X_test2 = np.hstack([X_test, X_test_cluster_dist])
    
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=44)
    
    clf = LogisticRegression()
    param_grid = {'C' : [2.0 ** i for i in range(-5, 10)]}
    
    gridSearch = GridSearchCV(estimator=clf, param_grid=param_grid, cv=kfold)
    gridSearch.fit(X_train2, y_train)
    
    print("n_clusters = %d \nC = %0.3f \nAccuracy: %0.3f (+/- %0.3f)\n" % (n_clusters,
                                                                           gridSearch.best_params_['C'],
                                                                           gridSearch.best_score_,
                                                                           gridSearch.cv_results_['std_test_score'][gridSearch.best_index_] * 2))
    if best_score < gridSearch.best_score_:
        best_score = gridSearch.best_score_
        best_n_clusters = n_clusters

print("\n\nbest_n_clusters = %d" % best_n_clusters)

n_clusters = 2 
C = 0.062 
Accuracy: 0.799 (+/- 0.113)

n_clusters = 3 
C = 0.062 
Accuracy: 0.799 (+/- 0.113)

n_clusters = 4 
C = 0.031 
Accuracy: 0.799 (+/- 0.109)

n_clusters = 5 
C = 1.000 
Accuracy: 0.807 (+/- 0.083)

n_clusters = 6 
C = 1.000 
Accuracy: 0.807 (+/- 0.080)

n_clusters = 7 
C = 0.500 
Accuracy: 0.805 (+/- 0.074)

n_clusters = 8 
C = 2.000 
Accuracy: 0.809 (+/- 0.090)

n_clusters = 9 
C = 2.000 
Accuracy: 0.808 (+/- 0.091)

n_clusters = 10 
C = 2.000 
Accuracy: 0.811 (+/- 0.086)

n_clusters = 11 
C = 2.000 
Accuracy: 0.808 (+/- 0.103)

n_clusters = 12 
C = 0.500 
Accuracy: 0.810 (+/- 0.074)

n_clusters = 13 
C = 4.000 
Accuracy: 0.810 (+/- 0.105)

n_clusters = 14 
C = 2.000 
Accuracy: 0.816 (+/- 0.083)

n_clusters = 15 
C = 64.000 
Accuracy: 0.811 (+/- 0.067)

n_clusters = 16 
C = 2.000 
Accuracy: 0.819 (+/- 0.071)

n_clusters = 17 
C = 4.000 
Accuracy: 0.823 (+/- 0.091)

n_clusters = 18 
C = 2.000 
Accuracy: 0.818 (+/- 0.066)

n_clusters = 19 
C = 2.000 
Accuracy: 

Results are better than baseline: ** 0.832 (+/- 0.060) ** (n_clusters = 39, C = 4.000).

Now test our model on different splitting to make sure, that we didn't overfit.

In [8]:
kmeans = KMeans(n_clusters=39, random_state=44)

X_train_cluster_dist = kmeans.fit_transform(X_train)
X_test_cluster_dist = kmeans.transform(X_test)

scaler_cluster_dist = StandardScaler()

X_train_cluster_dist = scaler_cluster_dist.fit_transform(X_train_cluster_dist)
X_test_cluster_dist = scaler_cluster_dist.transform(X_test_cluster_dist)

X_train2 = np.hstack([X_train, X_train_cluster_dist])
X_test2 = np.hstack([X_test, X_test_cluster_dist])

kfold = KFold(n_splits=10, shuffle=True, random_state=42) # random_state = 42, before used 44

clf = LogisticRegression(C = 4.0)

scores = cross_val_score(clf, X_train2, y_train, cv=kfold)

print("Accuracy: %0.3f (+/- %0.3f)\n" % (scores.mean(), scores.std() * 2))

Accuracy: 0.820 (+/- 0.097)



This result is within the limits of error.

Make a submission to Kaggle.

In [9]:
clf.fit(X_train2, y_train)
y_pred = clf.predict(X_test2)
ans = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived':y_pred})
ans.to_csv("ans.csv", index=False)

The result is 0.77990. It is within the limits of error too.